In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path

In [3]:
BASE_DIR = Path.cwd().parent

returns = pd.read_csv(
    BASE_DIR /'data'/'processed'/'daily_returns.csv',
    index_col = 0,
    parse_dates = True
    
)

returns.head()

,HDFCBANK,ICICIBANK,INFY,NIFTY50,NIFTYMIDCAP50,RELIANCE,TCS
Date,,,,,,,
2020-01-07,0.015710,-0.005340,-0.014931,0.004982,0.007602,0.015267,0.002451
2020-01-08,-0.002621,0.005816,-0.013416,-0.002293,0.002606,-0.007539,0.022148
2020-01-09,0.011152,0.037962,0.012935,0.015722,0.016768,0.022770,-0.018302
2020-01-10,0.008849,-0.011136,0.014464,0.003342,0.004522,-0.000226,-0.000361
2020-01-13,0.002569,-0.003059,0.046843,0.005918,0.008185,-0.002555,-0.010536


In [4]:
event_date = '2020-03-24'

In [5]:
estimation_start = '2020-01-01'
estimation_end = '2020-03-28'

In [6]:
event_window = returns.loc["2020-03-16":"2020-04-10"]

In [7]:
market_return = returns['NIFTY50']

stocks = ['HDFCBANK','ICICIBANK','INFY','RELIANCE','TCS',]

abnormal_returns = pd.DataFrame(index=event_window.index)
for stock in stocks:
    # Estimation data
    y = returns.loc[estimation_start:estimation_end, stock]
    X = returns.loc[estimation_start:estimation_end, "NIFTY50"]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    
    # Predict expected returns during event window
    expected = model.predict(
        sm.add_constant(event_window["NIFTY50"])
    )
    
    # Abnormal return = actual - expected
    abnormal_returns[stock] = event_window[stock] - expected

In [8]:
abnormal_returns.head()

,HDFCBANK,ICICIBANK,INFY,RELIANCE,TCS
Date,,,,,
2020-03-16,0.006656,-0.008689,-0.026339,0.010354,-0.007117
2020-03-17,-0.000686,-0.061458,-0.029154,0.022182,-0.005723
2020-03-18,-0.052212,0.035491,0.012006,0.028294,0.037744
2020-03-19,0.044305,-0.017378,0.040491,-0.025075,0.012898
2020-03-20,-0.067413,-0.046179,0.014423,0.034195,0.052393


In [9]:
CAR = abnormal_returns.cumsum()

CAR.head()


,HDFCBANK,ICICIBANK,INFY,RELIANCE,TCS
Date,,,,,
2020-03-16,0.006656,-0.008689,-0.026339,0.010354,-0.007117
2020-03-17,0.005970,-0.070147,-0.055493,0.032536,-0.012840
2020-03-18,-0.046243,-0.034656,-0.043487,0.060830,0.024904
2020-03-19,-0.001938,-0.052034,-0.002996,0.035754,0.037802
2020-03-20,-0.069351,-0.098214,0.011427,0.069949,0.090195


In [10]:
PROCESSED_DIR = BASE_DIR / "data" / "processed"

abnormal_returns.to_csv(PROCESSED_DIR / "abnormal_returns.csv")
CAR.to_csv(PROCESSED_DIR / "CAR.csv")
